# Setting

In [0]:
!pip install kaggle
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"young917","key":"38554d8ee11a16273014fb43b7fdc0a1"}'}

In [0]:
%ls -1ha kaggle.json

kaggle.json


In [0]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
# Permission Warning 이 일어나지 않도록 
!chmod 600 ~/.kaggle/kaggle.json
# 본인이 참가한 모든 대회 보기 
!kaggle competitions list

ref                                               deadline             category            reward  teamCount  userHasEntered  
------------------------------------------------  -------------------  ---------------  ---------  ---------  --------------  
digit-recognizer                                  2030-01-01 00:00:00  Getting Started  Knowledge       3198           False  
titanic                                           2030-01-01 00:00:00  Getting Started  Knowledge      23092           False  
house-prices-advanced-regression-techniques       2030-01-01 00:00:00  Getting Started  Knowledge       5286            True  
connectx                                          2030-01-01 00:00:00  Getting Started  Knowledge        394           False  
nlp-getting-started                               2030-01-01 00:00:00  Getting Started      Kudos       1670            True  
competitive-data-science-predict-future-sales     2020-12-31 23:59:00  Playground           Kudos       7112   

In [0]:
!kaggle competitions download -c nlp-getting-started

  0% 0.00/965k [00:00<?, ?B/s]
100% 965k/965k [00:00<00:00, 60.5MB/s]
  0% 0.00/22.2k [00:00<?, ?B/s]
100% 22.2k/22.2k [00:00<00:00, 19.5MB/s]
  0% 0.00/411k [00:00<?, ?B/s]
100% 411k/411k [00:00<00:00, 98.7MB/s]


In [0]:
from google.colab import drive
drive.mount('/content/gdrive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/


# **Data Load**

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', 200)
import spacy

plt.rcParams["figure.figsize"] = (15,5)

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  


In [0]:
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')
print(test)

         id  ...                                                                                                                                         text
0     0      ...  Just happened a terrible car crash                                                                                                         
1     2      ...  Heard about #earthquake is different cities, stay safe everyone.                                                                           
2     3      ...  there is a forest fire at spot pond, geese are fleeing across the street, I cannot save them all                                           
3     9      ...  Apocalypse lighting. #Spokane #wildfires                                                                                                   
4     11     ...  Typhoon Soudelor kills 28 in China and Taiwan                                                                                              
...   ..     ...                                    

### final_text : keyword + text

In [0]:
train['keyword'].fillna('', inplace=True)
train['final_text'] = train['keyword'] + ' ' + train['text'] 
test['keyword'].fillna('', inplace=True)
test['final_text'] = test['keyword'] + ' ' + test['text'] 

In [0]:
train.head(100)

,id,keyword,location,text,target,final_text
0,1,,NaN,Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all,1,Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all
1,4,,NaN,Forest fire near La Ronge Sask. Canada,1,Forest fire near La Ronge Sask. Canada
2,5,,NaN,All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected,1,All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected
3,6,,NaN,"13,000 people receive #wildfires evacuation orders in California",1,"13,000 people receive #wildfires evacuation orders in California"
4,7,,NaN,Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school,1,Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school
5,8,,NaN,#RockyFire Update => California Hwy. 20 closed in both directions due to Lake County fire - #CAfire #wildfires,1,#RockyFire Update => California Hwy. 20 closed in both directions due to Lake County fire - #CAfire #wildfires
6,10,,NaN,"#flood #disaster Heavy rain causes flash flooding of streets in Manitou, Colorado Springs areas",1,"#flood #disaster Heavy rain causes flash flooding of streets in Manitou, Colorado Springs areas"
7,13,,NaN,I'm on top of the hill and I can see a fire in the woods...,1,I'm on top of the hill and I can see a fire in the woods...
8,14,,NaN,There's an emergency evacuation happening now in the building across the street,1,There's an emergency evacuation happening now in the building across the street
9,15,,NaN,I'm afraid that the tornado is coming to our area...,1,I'm afraid that the tornado is coming to our area...


#### Data Cleaning

In [0]:
from nltk.corpus import stopwords
from nltk.util import ngrams
import re
from nltk.tokenize import word_tokenize
import gensim
import string
from tqdm import tqdm

import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow_hub as hub

In [0]:
import nltk

nltk.download('stopwords')
stop=set(stopwords.words('english'))

punctuation = string.punctuation

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


##### train과 test set에서 URL, html, 구두점을 삭제한다.

In [0]:
df=pd.concat([train,test])
df.shape

(10876, 6)

In [0]:
import re

# remove http
def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'',text)

df['final_text']=df['final_text'].apply(lambda x : remove_URL(x))

# remove HTML tags
def remove_html(text):
    html=re.compile(r'<.*?>')
    return html.sub(r'',text)
df['final_text']=df['final_text'].apply(lambda x : remove_html(x))

# remove punctuations
def remove_punct(text):
    table=str.maketrans('','',string.punctuation)
    return text.translate(table)
df['final_text']=df['final_text'].apply(lambda x : remove_punct(x))

### Remove Mislabeled
https://www.kaggle.com/rai555/real-or-not-nlp-huggingface-bert-pytorch

## **Model**

In [0]:
!wget --quiet https://raw.githubusercontent.com/tensorflow/models/master/official/nlp/bert/tokenization.py

In [0]:
!pip install sentencepiece

     |████████████████████████████████| 1.1MB 9.3MB/s 


#### Encoding

In [0]:
# text -> tokens with padding
def bert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [0]:
def build_model(bert_layer, max_len=512):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")

    _, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
    clf_output = sequence_output[:, 0, :]
    out = Dense(1, activation='sigmoid')(clf_output)
    
    model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    model.compile(Adam(lr=2e-6), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [0]:
%%time
module_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1"
bert_layer = hub.KerasLayer(module_url, trainable=True)

CPU times: user 24.1 s, sys: 4.31 s, total: 28.4 s
Wall time: 39.3 s


In [0]:
import tokenization

vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)

In [0]:
train = df[:-3263]
test = df[-3263:]

In [0]:
train_input = bert_encode(train.final_text.values, tokenizer, max_len=160)
test_input = bert_encode(test.final_text.values, tokenizer, max_len=160)
train_labels = train.target.values

#### Build Model

In [0]:
model = build_model(bert_layer, max_len=160)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 160)]        0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 160)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 160)]        0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        [(None, 1024), (None 335141889   input_word_ids[0][0]             
                                                                 input_mask[0][0]             

### With adding keyword

In [0]:
%%time
train_history = model.fit(
    train_input, train_labels,
    validation_split=0.2,
    epochs=3,
    batch_size=2
)

Epoch 1/3
3045/3045 [==============================] - 522s 171ms/step - loss: 0.4332 - accuracy: 0.8143 - val_loss: 0.4044 - val_accuracy: 0.8365
Epoch 2/3
3045/3045 [==============================] - 521s 171ms/step - loss: 0.2808 - accuracy: 0.8878 - val_loss: 0.4215 - val_accuracy: 0.8194
Epoch 3/3
3045/3045 [==============================] - 520s 171ms/step - loss: 0.1392 - accuracy: 0.9527 - val_loss: 0.4660 - val_accuracy: 0.8175
CPU times: user 17min 2s, sys: 8min 18s, total: 25min 21s
Wall time: 26min 33s


### Without adding keyword

In [0]:
%%time
train_history = model.fit(
    train_input, train_labels,
    validation_split=0.2,
    epochs=3,
    batch_size=2
)

Epoch 1/3
3045/3045 [==============================] - 1747s 574ms/step - loss: 0.4356 - accuracy: 0.8077 - val_loss: 0.3797 - val_accuracy: 0.8424
Epoch 2/3
3045/3045 [==============================] - 1737s 571ms/step - loss: 0.2723 - accuracy: 0.8943 - val_loss: 0.3924 - val_accuracy: 0.8313
Epoch 3/3
3045/3045 [==============================] - 1738s 571ms/step - loss: 0.1293 - accuracy: 0.9535 - val_loss: 0.4605 - val_accuracy: 0.8299
CPU times: user 50min 35s, sys: 32min 15s, total: 1h 22min 50s
Wall time: 1h 27min 24s


In [0]:
model.save('Bertmodel.h5')

In [0]:
cp './Bertmodel.h5' '/content/gdrive/My Drive/Data Mining Project/Bertmodel.h5'

## Submission

In [0]:
submission=pd.read_csv('./sample_submission.csv')

In [0]:
test_pred = model.predict(test_input)
submission['target'] = test_pred.round().astype(int)
submission.to_csv('Bert+keyword_submission.csv', index=False)

In [0]:
submission.head()

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1


In [0]:
!cp './Bert+keyword_submission.csv' '/content/gdrive/My Drive/Data Mining Project/Bert+keyword_submission.csv'

In [0]:
submission

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1
...,...,...
3258,10861,0
3259,10865,1
3260,10868,1
3261,10874,1
